# Chapter 10 : 기호 수식 다루기

## 10.1 컴퓨터 대수 시스템으로 정확한 도함수 구하기

In [44]:
from math import sin
def f(x):
    return (3*x**2 + x) * sin(x)

## 10.2 대수식 모델링하기

### 10.2.3 수식 트리를 파이썬 언어로 번역하기

In [45]:
class Power():
    def __init__(self, base, exponenet):
        self.base = base
        self.exponent = exponenet

In [46]:
class Number():
    def __init__(self, number):
        self.number = number

class Variable():
    def __init__(self, symbol):
        self.symbol = symbol

This represents $x^2$:

In [47]:
Power(Variable("x"),Number(2))

In [48]:
class Product():
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2

This represents $3x^2$:

In [49]:
Product(Number(3),Power(Variable("x"),Number(2)))

In [50]:
class Sum():
    def __init__(self, *exps): #<1>
        self.exps = exps

class Function(): #<2>
    def __init__(self,name):
        self.name = name

class Apply(): #<3>
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument

f_expression = Product( #<4>
                Sum(
                    Product(
                        Number(3),
                        Power(
                            Variable("x"),
                            Number(2))), 
                    Variable("x")), 
                Apply(
                    Function("sin"),
                    Variable("x")))

This represents $\cos(x^3 + -5)$:

In [51]:
Apply(Function("cos"),Sum(Power(Variable("x"),Number("3")), Number(-5)))

### 10.2.4 연습문제

10.1 수식 $\ln(y^z)$ 을 그려라.

10.2 자연로그는 파이썬 함수 math.log로 계산한다고 가정하자. 이 수식을 파이썬 함수로 작성하고 원소와 컴비네이터로 만든 자료구조로도 작성하라.

In [52]:
from math import log
def f(y,z):
    return log(y**z)

In [53]:
Apply(Function("ln"), Power(Variable("y"), Variable("z")))

10.4 한 수식을 다른 수식으로 나누는 나눗셈(Quotient) 컴비네이터를 구현하라. 
$$\frac{a+b}{2}$$

In [54]:
class Quotient():
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator

Here's the representation of $(a+b)/2$:

In [55]:
Quotient(Sum(Variable("a"),Variable("b")),Number(2))

10.5 한 수식과 다른 수식의 뺄셈을 나타내는 차(Difference) 컴비네이터를 구현하라. $b^2 - 4ac$를 컴비네이터로 표현하라.

In [56]:
class Difference():
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2

In [57]:
Difference(
    Power(Variable('b'),Number(2)),
    Product(Number(4),Product(Variable('a'), Variable('c')))
    )

10.6 수식에 음의 기호를 붙이는 부정(Negative) 컴비네이터를 구현하라. 예를 들어 $x^2 + y$의 부정은 $-(x^2 + y)$이다. 새로운 컴비네이터를 사용해 나타내어라.

In [58]:
class Negative():
    def __init__(self,exp):
        self.exp = exp

In [59]:
Negative(Sum(Power(Variable("x"),Number(2)),Variable("y")))

10.7 제곱근을 나타내는 Sqrt라는 함수를 추가하고 이 함수를 사용해 다음 수식을 코드화하라.$$\frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$$

In [60]:
A = Variable('a')
B = Variable('b')
C = Variable('c')
Sqrt = Function('sqrt')

In [61]:
Quotient(Sum(Negative(B),Apply(Sqrt, 
            Difference(Power(B,Number(2)),Product(Number(4), Product(A,C))))),Product(Number(2), A))

## 10.3 기호 수식 동작하게 하기

In [62]:
def distinct_variables(exp):
    if isinstance(exp, Variable):
        return set(exp.symbol)
    elif isinstance(exp, Number):
        return set()
    elif isinstance(exp, Sum):
        return set().union(*[distinct_variables(exp) for exp in exp.exps])
    elif isinstance(exp, Product):
        return distinct_variables(exp.exp1).union(distinct_variables(exp.exp2))
    elif isinstance(exp, Power):
        return distinct_variables(exp.base).union(distinct_variables(exp.exponent))
    elif isinstance(exp, Apply):
        return distinct_variables(exp.argument)
    else:
        raise TypeError("Not a valid expression.")

In [63]:
distinct_variables(Variable("z"))

{'z'}

In [64]:
distinct_variables(Number(3))

set()

In [65]:
distinct_variables(f_expression)

{'x'}

### 10.3.2 수식 값 구하기

In [66]:
from abc import ABC, abstractmethod

class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass

In [67]:
class Number(Expression):
    def __init__(self,number):
        self.number = number
    def evaluate(self, **bindings):
        return self.number
    
class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol
    def evaluate(self, **bindings):
        try:
            return bindings[self.symbol]
        except:
            raise KeyError("Variable '{}' is not bound.".format(self.symbol))
            
class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)

In [68]:
Product(Variable("x"), Variable("y")).evaluate(x=2,y=5)

10

In [69]:
import math
from math import sin, cos, log

_function_bindings = {
    "sin": math.sin,
    "cos": math.cos,
    "ln": math.log
}

class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument
    def evaluate(self, **bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))

In [70]:
class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])
    
class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent
    def evaluate(self, **bindings):
        return self.base.evaluate(**bindings) ** self.exponent.evaluate(**bindings)
    
class Difference(Expression):
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) - self.exp2.evaluate(**bindings)
    
class Quotient(Expression):
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator
    def evaluate(self, **bindings):
        return self.numerator.evaluate(**bindings) / self.denominator.evaluate(**bindings)

In [71]:
f_expression = Product(
                Sum(Product(Number(3),Power(Variable("x"),Number(2))), Variable("x")), Apply(Function("sin"),Variable("x")))

In [72]:
f_expression.evaluate(x=5)

-76.71394197305108

In [73]:
from math import sin
def f(x):
    return (3*x**2 + x) * sin(x)

f(5)

-76.71394197305108

### 10.3.3 수식 전개하기

In [74]:
class Expression(ABC):
    @abstractmethod
    def evaluate(self, **bindings):
        pass
    @abstractmethod
    def expand(self):
        pass
    
    # Printing expressions legibly in REPL (See first mini project in 2.4)
    @abstractmethod
    def display(self):
        pass
    def __repr__(self):
        return self.display()

In [75]:
class Sum(Expression):
    def __init__(self, *exps):
        self.exps = exps
    def evaluate(self, **bindings):
        return sum([exp.evaluate(**bindings) for exp in self.exps])
    def expand(self):
        return Sum(*[exp.expand() for exp in self.exps])
    def display(self):
        return "Sum({})".format(",".join([e.display() for e in self.exps]))
    
class Product(Expression):
    def __init__(self, exp1, exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) * self.exp2.evaluate(**bindings)
    def expand(self):
        expanded1 = self.exp1.expand()
        expanded2 = self.exp2.expand()
        if isinstance(expanded1, Sum):
            return Sum(*[Product(e,expanded2).expand() for e in expanded1.exps])
        elif isinstance(expanded2, Sum):
            return Sum(*[Product(expanded1,e) for e in expanded2.exps])
        else:
            return Product(expanded1,expanded2)
    def display(self):
        return "Product({},{})".format(self.exp1.display(),self.exp2.display())
        
class Difference(Expression):
    def __init__(self,exp1,exp2):
        self.exp1 = exp1
        self.exp2 = exp2
    def evaluate(self, **bindings):
        return self.exp1.evaluate(**bindings) - self.exp2.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Difference({},{})".format(self.exp1.display(), self.exp2.display())
    
class Quotient(Expression):
    def __init__(self,numerator,denominator):
        self.numerator = numerator
        self.denominator = denominator
    def evaluate(self, **bindings):
        return self.numerator.evaluate(**bindings) / self.denominator.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Quotient({},{})".format(self.numerator.display(),self.denominator.display())
    
class Negative(Expression):
    def __init__(self,exp):
        self.exp = exp
    def evaluate(self, **bindings):
        return - self.exp.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Negative({})".format(self.exp.display())
    
class Number(Expression):
    def __init__(self,number):
        self.number = number
    def evaluate(self, **bindings):
        return self.number
    def expand(self):
        return self
    def display(self):
        return "Number({})".format(self.number)
    
class Power(Expression):
    def __init__(self,base,exponent):
        self.base = base
        self.exponent = exponent
    def evaluate(self, **bindings):
        return self.base.evaluate(**bindings) ** self.exponent.evaluate(**bindings)
    def expand(self):
        return self
    def display(self):
        return "Power({},{})".format(self.base.display(),self.exponent.display())
    
class Variable(Expression):
    def __init__(self,symbol):
        self.symbol = symbol
    def evaluate(self, **bindings):
        return bindings[self.symbol]
    def expand(self):
        return self
    def display(self):
        return "Variable(\"{}\")".format(self.symbol)
    
class Function():
    def __init__(self,name,make_latex=None):
        self.name = name
        self.make_latex = make_latex
    def latex(self,arg_latex):
        if self.make_latex:
            return self.make_latex(arg_latex)
        else:
            return " \\operatorname{{ {} }} \\left( {} \\right)".format(self.name, arg_latex)
  
class Apply(Expression):
    def __init__(self,function,argument):
        self.function = function
        self.argument = argument
    def evaluate(self, **bindings):
        return _function_bindings[self.function.name](self.argument.evaluate(**bindings))
    def expand(self):
        return Apply(self.function, self.argument.expand())
    def display(self):
        return "Apply(Function(\"{}\"),{})".format(self.function.name, self.argument.display())

In [76]:
Y = Variable('y')
Z = Variable('z')
A = Variable('a')
B = Variable('b')
Product(Sum(A,B),Sum(Y,Z))

Product(Sum(Variable("a"),Variable("b")),Sum(Variable("y"),Variable("z")))

In [77]:
Product(Sum(A,B),Sum(Y,Z)).expand()

Sum(Sum(Product(Variable("a"),Variable("y")),Product(Variable("a"),Variable("z"))),Sum(Product(Variable("b"),Variable("y")),Product(Variable("b"),Variable("z"))))

### 10.3.4 연습문제

10.9 주어진 수식이 특정 변수를 포함하는지 여부를 확인하는 함수 contains(expression, variable)을 작성하라.

In [78]:
def contains(exp, var):
    if isinstance(exp, Variable):
        return exp.symbol == var.symbol
    elif isinstance(exp, Number):
        return False
    elif isinstance(exp, Sum):
        return any([contains(e,var) for e in exp.exps])
    elif isinstance(exp, Product):
        return contains(exp.exp1,var) or contains(exp.exp2,var)
    elif isinstance(exp, Power):
        return contains(exp.base, var) or contains(exp.exponent, var)
    elif isinstance(exp, Apply):
        return contains(exp.argument, var)
    else:
        raise TypeError("Not a valid expression.")

10.10 수식을 인자로 받아서 (sin이나 ln과 같이) 이름이 붙은 함수를 리턴하는 함수 distinct_functions를 작성하라.

In [79]:
def distinct_functions(exp):
    if isinstance(exp, Variable):
        return set()
    elif isinstance(exp, Number):
        return set()
    elif isinstance(exp, Sum):
        return set().union(*[distinct_functions(exp) for exp in exp.exps])
    elif isinstance(exp, Product):
        return distinct_functions(exp.exp1).union(distinct_functions(exp.exp2))
    elif isinstance(exp, Power):
        return distinct_functions(exp.base).union(distinct_functions(exp.exponent))
    elif isinstance(exp, Apply):
        return set([exp.function.name]).union(distinct_functions(exp.argument))
    else:
        raise TypeError("Not a valid expression.")

10.11 수식을 입력으로 받아 Sum을 포함하고 있으면 True, 그렇지 않으면 False를 리턴하는 함수 contains_sum을 작성하라.

In [80]:
def contains_sum(exp):
    if isinstance(exp, Variable):
        return False
    elif isinstance(exp, Number):
        return False
    elif isinstance(exp, Sum):
        return True
    elif isinstance(exp, Product):
        return contains_sum(exp.exp1) or contains_sum(exp.exp2)
    elif isinstance(exp, Power):
        return contains_sum(exp.base) or contains_sum(exp.exponent)
    elif isinstance(exp, Apply):
        return contains_sum(exp.argument)
    else:
        raise TypeError("Not a valid expression.")

## 10.4 함수의 도함수 구하기

## 10.5 도함수 취하는 과정 자동화하기

### 10.5.1 기호 수식의 도함수 구하는 법 구현하기

In [82]:
from expressions import *

In [83]:
Product(Power(Variable("x"),Number(2)),Apply(Function("sin"),Variable("y")))

Product(Power(Variable("x"),Number(2)),Apply(Function("sin"),Variable("y")))

In [84]:
Sum(Variable("x"),Variable("c"),Number(1)).derivative(Variable("x"))

Sum(Number(1),Number(0),Number(0))

### 10.5.2 곱의 미분법과 합성함수의 미분법 구현하기

In [85]:
Product(Variable("c"),Variable("x")).derivative(Variable("x"))

Product(Variable("c"),Number(1))

In [86]:
Apply(Function("sin"),Power(Variable("x"),Number(2))).derivative(x)

Product(Product(Number(1),Product(Number(2),Power(Variable("x"),Number(1)))),Apply(Function("cos"),Power(Variable("x"),Number(2))))

### 10.5.3 거듭제곱의 미분법 구현하기

In [89]:
f_expression = Product( 
                Sum(Product(Number(3),Power(Variable("x"),Number(2))), Variable("x")), 
                Apply(Function("sin"),Variable("x")))

In [90]:
f_expression.derivative(x)

Sum(Product(Sum(Product(Number(3),Product(Number(1),Product(Number(2),Power(Variable("x"),Number(1))))),Number(1)),Apply(Function("sin"),Variable("x"))),Product(Sum(Product(Number(3),Power(Variable("x"),Number(2))),Variable("x")),Product(Number(1),Apply(Function("cos"),Variable("x")))))

## 10.6 기호적으로 함수 적분하기

### 10.6.2 SymPy 라이브러리 소개

In [91]:
from sympy import *
from sympy.core.core import *
Mul(Symbol('y'), Add(3, Symbol('x')))

y*(x + 3)

In [92]:
y = Symbol('y')

In [93]:
x = Symbol('x')

In [94]:
y*(3+x)

y*(x + 3)

In [95]:
y*(3+x).subs(x,1)

4*y

In [96]:
(x**2).diff(x)

2*x

In [97]:
(3*x**2).integrate(x)

x**3

### 10.6.3 연습문제

10.22 $f(x) = 0$의 부정적분을 구하라. SymPy로 구한 답을 확인하되 SymPy가 적분상수를 자동으로 포함하지 않음을 주의하라.

In [99]:
Integer(0).integrate(x)

0

10.23  $x\cdot \cos(x)$의 부정적분을 구하라.

In [100]:
(x*cos(x)).integrate(x)

x*sin(x) + cos(x)

10.24 $x^2$의 부정적분을 구하라.

In [101]:
(x**2).integrate(x)

x**3/3